# Práctico 3.A - Regresión Lineal

## Diabetes dataset

Un dataset provisto por sklearn utilizado para el entrenamiento de distintos algoritmos de machine learning, contiene información acerca de 442 pacientes con diabetes, y una medida de la progresión de la enfermedad al cabo de un año desde el baseline.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
from sklearn.datasets import load_diabetes

data_bulk = load_diabetes(return_X_y=False)
df = pd.DataFrame(data_bulk.data, columns = data_bulk.feature_names)
df['target'] = data_bulk.target

In [ ]:
data_bulk

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.head()

NOTA: Los datos están estandarizados, interpretarlos directamente no tiene un sentido real.

#### Descripción de los datos

* age - age in years
* sex
* bmi - body mass index
* bp - average blood pressure
* s1 - tc, total serum cholesterol
* s2 - ldl, low-density lipoproteins
* s3 - hdl, high-density lipoproteins
* s4 - tch, total cholesterol / HDL
* s5 - ltg, possibly log of serum triglycerides level
* s6 - glu, blood sugar level
* target - Column 11 is a quantitative measure of disease progression one year after baseline

fuentes:
* [Diabetes Dataset - sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes)
* [Diabetes data](https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)

In [ ]:
df.describe().apply(lambda s: s.apply('{0:.5f}'.format))

In [ ]:
cmap = sns.diverging_palette(220, 20, sep=20, as_cmap=True)
plt.figure(figsize=(17,9))
sns.heatmap(df.corr(), annot=True,cmap=cmap, center=0).set_title("Correlation Heatmap", fontsize=16)

In [ ]:
df.plot.scatter(x = "s1", y = "s2")
#df = df.drop(columns ="s2")


In [ ]:
df

### Particionar datos en train y test en proporción 2/3 y 1/3, respectivamente

In [ ]:
from sklearn.model_selection import train_test_split
random_state = 17

X, Y = df.loc[:, df.columns != 'target'], df["target"]
X_train, X_test, Y_train, Y_test = train_test_split(
    X,
    Y,
    test_size=(1.0/3),
    random_state=random_state)

### Entrenamiento

* [Linear Model - sklearn](https://scikit-learn.org/stable/modules/linear_model.html)

In [ ]:
from sklearn import linear_model

regr = linear_model.LinearRegression()
regr.fit(X_train, Y_train)

Coeficientes

In [ ]:
regr.coef_

In [ ]:
regr.intercept_ #termino independiente

In [ ]:
feature = "bmi"

Y_pred = regr.predict(X_test)

sorted_indices = X_test[feature].argsort()
X_test_sorted = X_test[feature].iloc[sorted_indices]
Y_pred_sorted = Y_pred[sorted_indices]

plt.figure(figsize=(8, 6))
plt.scatter(X_test[feature], Y_test, color="blue", label="Datos reales")

plt.plot(X_test_sorted, Y_pred_sorted, color="red", label="Línea de regresión")

plt.title(f"Regresión lineal: {feature} vs target")
plt.xlabel(feature)
plt.ylabel("Target")
plt.legend()
plt.grid(True)
plt.show()


Estamos evaluando visualmente cómo nuestra regresión lineal multivariable predice el target en función de una sola variable (bmi). Aunque estamos graficando respecto a bmi, el modelo usó también las otras features (age, bp, etc.) para hacer esas predicciones.

La línea roja no es una regresión simple de target ~ bmi, sino una curva de predicciones del modelo completo ordenadas por bmi. Como el modelo usó muchas otras variables además de bmi, la predicción puede cambiar fuertemente incluso si bmi es similar, si las demás variables cambian mucho.

In [ ]:
feature = "bmi"

bmi_range = np.linspace(X_test['bmi'].min(), X_test['bmi'].max(), 100)

X_bmi_train = X_train[['bmi']]
X_bmi_test = X_test[['bmi']]
regr_bmi = linear_model.LinearRegression()
regr_bmi.fit(X_bmi_train, Y_train)
Y_bmi_pred_range = regr_bmi.predict(bmi_range.reshape(-1, 1))

Y_pred_full_model = regr.predict(X_test)  # Predicciones con todas las variables
sorted_indices = X_test['bmi'].argsort()
X_test_sorted_by_bmi = X_test['bmi'].iloc[sorted_indices]
Y_pred_sorted_by_bmi = Y_pred_full_model[sorted_indices]

plt.figure(figsize=(10, 6))
plt.scatter(X_test['bmi'], Y_test, color="blue", alpha=0.5, label="Datos reales")

plt.plot(bmi_range, Y_bmi_pred_range, color="green", label="Regresión solo con 'bmi'")
plt.plot(X_test_sorted_by_bmi, Y_pred_sorted_by_bmi, color="red", label="Modelo completo (ordenado por 'bmi')")

plt.title("Comparación: Modelo con solo 'bmi' vs. Modelo completo (ordenado por 'bmi')")
plt.xlabel('bmi')
plt.ylabel('Target')
plt.legend()
plt.grid(True)
plt.show()

# Modelo verde

- Es el modelo más simple posible.

- Se ajusta lo mejor que puede usando solo la variable bmi.

- Su forma recta indica que supone una relación lineal directa entre bmi y el target.

# Conclusiones

1. El modelo con solo bmi (verde) capta la tendencia global, pero no puede adaptarse a la complejidad de los datos.

2. El modelo completo (rojo) puede ajustarse más a los datos, pero muestra que las predicciones varían bastante incluso si bmi no lo hace mucho —lo que indica que otras variables son importantes.

3. La dispersión de puntos sugiere que bmi por sí solo no es suficiente para explicar bien el target.

4. Si la línea verde fuera muy parecida a la roja, significaría que bmi es casi todo lo que importa. Pero como no es así, ves que el modelo necesita otras variables para mejorar las predicciones.

In [ ]:
feature_names = X_train.columns.tolist()
coefficients = regr.coef_.tolist()

plt.figure(figsize=(10, 6))
plt.barh(feature_names, coefficients)
plt.xlabel('Coefficient Value')
plt.ylabel('Feature')
plt.title('Linear Regression Coefficients')
plt.grid(axis='x')
plt.show()

## Evaluación con métricas

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

y_pred = regr.predict(X_test)

$
   \LARGE\text{MAE} = \frac{1}{n} \sum_{i=1}^{n} \left(|\hat{y}_i - y_i|  \right)
   $

$
   \LARGE\text{MSE} = \frac{1}{n} \sum_{i=1}^{n} \left(\hat{y}_i - y_i \right)^2
   $

$ \LARGE R^2 = 1 - \frac{\sum_{i=1}^{n} \left( y_i - \hat{y}_i \right)^2}{\sum_{i=1}^{n} \left( y_i - \bar{y} \right)^2}
$

$ y_i \text{ son los valores reales}, \\
\hat{y}_i \text{ son los valores predichos},  \\
\bar{y} \text{ es la media de los valores reales},  \\
n \text{ es el número total de datos}.$

Coeficiente de determinacion: R-Cuadrado (R² o el coeficiente de determinación) es una medida estadística en un modelo de regresión que determina la proporción de varianza en la variable dependiente que puede ser explicada por la variable independiente. En otras palabras, r-cuadrado muestra lo bien que se ajustan los datos al modelo de regresión (la bondad del ajuste).

*es una medida que nos dice qué tan bien un modelo de regresión está explicando la variabilidad de los datos.*

In [ ]:
print("MAE")
print(f'(base regresssion) = {mean_absolute_error(Y_test, y_pred)}')
print("\n")

print("MSE")
print(f'(base regresssion) = {mean_squared_error(Y_test, y_pred)}')
print("\n")


print("R2-Score")
RSS = np.sum(np.square(Y_test - regr.predict(X_test))) #residual sum square
TSS = np.sum(np.square(Y_test - np.mean(Y_test))) #total sum square
R2 = 1 - RSS/TSS
print("R-squared score on training data (manual):", R2)

score = regr.score(X_test, Y_test)
print("R-squared score on training data:", score)


In [ ]:
predicted = pd.DataFrame(regr.predict(X_test), columns = ["predicted"])
plt.scatter(Y_test, predicted["predicted"])
plt.xlabel("target")
plt.ylabel("predicted")
plt.show()

## Houses price prediction

Este dataset contiene datos y precios de casas de King County, Washington. Los datos refieren a casas vendidas entre mayo de 2014 y mayo de 2015.

### Ejercicio 1
Cargue el dataset houses_data

### Ejercicio 2
Utilizando las herramientas vistas en el curso, realice un análisis exploratorio de los datos del dataframe y haga las modificaciones que crea necesarias.

### Ejercicio 3
Entrene al menos dos modelos de regresión distintos para predecir el precio. Recuerde utilizar alguna de las técnicas de particionamiento vistas en el curso.

### Ejercicio 4
Con las distintas métricas que vimos en clase, evalúe la performance de ambos modelos. Conclusiones.